In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import mean_squared_error

def rmse(y_te, y_p):
    return np.sqrt(mean_squared_error(y_te, y_p))

In [3]:
data = pd.read_pickle('data.pkl')
data = data[data.date_block_num > 11]

In [4]:
X = data[data.date_block_num < 34].drop(['item_cnt_month'], axis=1)
Y = data[data.date_block_num < 34]['item_cnt_month']
x = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [5]:
# create correlation matrix
corr = data.corr()

# store the heavily correlated columns
corr_set = set()

threshold = 0.5
# iterate over content and check where they exceed a given threshold
for row in corr.iterrows():
    row_name = row[0]
    for entry in row[1].iteritems():
        entry_name = entry[0]
        value = abs(entry[1])
        if (value > threshold) and (value < 1.0):
            corr_set.add(row_name)
            corr_set.add(entry_name)
            # print(f'{row_name} & {entry_name}')

# do not drop the label
corr_set.remove('item_cnt_month')

# remove it from the data column
for corr in corr_set:
    data.drop([corr], axis=1, inplace=True)

In [7]:
corr_set

{'category_device_for_platform',
 'date_avg_item_cnt_lag_1',
 'date_avg_item_cnt_lag_12',
 'date_avg_item_cnt_lag_2',
 'date_avg_item_cnt_lag_3',
 'date_avg_item_cnt_lag_6',
 'date_cat-device_avg_item_cnt_lag_1',
 'date_cat-device_avg_item_cnt_lag_12',
 'date_cat-device_avg_item_cnt_lag_2',
 'date_cat-device_avg_item_cnt_lag_3',
 'date_cat-device_avg_item_cnt_lag_6',
 'date_cat-medium-type_avg_item_cnt_lag_1',
 'date_cat-medium-type_avg_item_cnt_lag_12',
 'date_cat-medium-type_avg_item_cnt_lag_2',
 'date_cat-medium-type_avg_item_cnt_lag_3',
 'date_cat-medium-type_avg_item_cnt_lag_6',
 'date_cat-type_avg_item_cnt_lag_1',
 'date_cat-type_avg_item_cnt_lag_12',
 'date_cat-type_avg_item_cnt_lag_2',
 'date_cat-type_avg_item_cnt_lag_3',
 'date_cat-type_avg_item_cnt_lag_6',
 'date_cat_avg_item_cnt_lag_1',
 'date_cat_avg_item_cnt_lag_12',
 'date_cat_avg_item_cnt_lag_2',
 'date_cat_avg_item_cnt_lag_3',
 'date_cat_avg_item_cnt_lag_6',
 'date_city_avg_item_cnt_lag_1',
 'date_city_avg_item_cnt_lag_

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.72,
                                                    random_state=42)
lf = LinearRegression()
lf.fit(X_train, y_train)
prediction = lf.predict(X_test).clip(0,20)

shoved_data_rmse = rmse(y_test, prediction)

print(f"RMSE: {shoved_data_rmse}")

RMSE: 0.7977978573326405
